## In this notebook:

* single input dataframe 
* using all scores for predictors
* fully connected neural net
* training with dataloading and epochs
* aucuracy assessement of .75 on held-out test 

### Imports

In [1]:
#!pip install numpy 
#!pip install pandas
#!pip install torch
#!pip install sklearn

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.optim as optim

### Manipulating data
* Import csv we want
* Map our features to usuable integers
* Map our labels to a one hot encoding

In [2]:
#################################################### 
## pull in pickled dataframe:
df = pd.read_pickle("./pickled_df/all_scores.pkl")


# Split our data into two separate sets for training purposes
train, test = train_test_split(df, shuffle=False)

print(test.head())
print(train.shape, test.shape)

                     SIFT_score SIFT4G_score Polyphen2_HDIV_score  \
chr pos      ref alt                                                
20  62473533 A   G        0.011          0.0                0.972   
    62475476 A   C         0.03        0.008                  1.0   
    62952855 C   T        0.009        0.018                0.476   
    63361113 A   C        0.062        0.094                0.637   
    63361125 G   A        0.722        0.652                  0.0   

                     Polyphen2_HVAR_score LRT_score MutationTaster_score  \
chr pos      ref alt                                                       
20  62473533 A   G                  0.968  0.000000                    1   
    62475476 A   C                    1.0  0.004342             0.822704   
    62952855 C   T                  0.118  0.012156                    1   
    63361113 A   C                  0.102  0.357563             0.999525   
    63361125 G   A                    0.0  0.039676         

### DataLoader
* Implement the Dataset class supplied by PyTorch
  * Used Map style
  * API: https://pytorch.org/docs/stable/data.html#map-style-datasets
* Instanciate DataLoader using Dataset instance

In [3]:
class DataSetdbNSFP(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data.values.astype('float'))
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
            target = self.data[index][-1]
            data_val = self.data[index] [:-1]
            return data_val,target
        
train_dataset = DataSetdbNSFP(train)
valid_dataset = DataSetdbNSFP(train)
test_dataset = DataSetdbNSFP(test)

train_loader = DataLoader(train_dataset, batch_size=59, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=59, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=59, shuffle=True)


### Model
* This model is a simple implementation
* Based on https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [4]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # In: nb of features, out: nb of examples
        self.fc = nn.Linear(27, 1000)
        # In: nb of examples, out: nb of predictions
        self.fc2 = nn.Linear(1000, 2)

    def forward(self, x):
        x = x.view((-1, 27))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        # Softmax to get the actual labels
        return F.softmax(h, dim=1)  ## Note: can potentially use this to get confidence of category predicted 
    
model = Model()
# if cuda:
#     model.cuda()

# Optimizer based on Adam algorithm, uses a slightly lower rate
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Training
* Run a fixed number of epochs
* After each epoch, run the validation set to adjust learning
* Print results
* Based on example: https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [5]:
EPOCHS = 20

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        print(data.shape)
        print(target.shape)
        
#         if cuda:
#             data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        y_pred = model(data) 
        
        # Calculate and appy loss
        loss = F.cross_entropy(y_pred, target.long())
        loss.backward()
        optimizer.step()

        print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1,
            EPOCHS,
            batch_idx * len(data), 
            len(train_loader.dataset),
            100. * batch_idx / len(train_loader), 
            loss.cpu().data.item()))
    
    # NOTE: The Variable class is a wrapper class around a tensor with the added functionalities of back propagation
    # For x, take all cols of datafram except the last one
    evaluate_x = Variable(valid_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
    print(evaluate_x)
    # For y, take last col
    evaluate_y = Variable(valid_loader.dataset.data[:, -1:])
#     if cuda:
#         evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()
    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    
    y_labels = torch.flatten(evaluate_y.data)
    print('Predictions:', pred)
    print('Actual values:', y_labels)
    d = pred.eq(y_labels).cpu()
    accuracy = d.sum().item()*1./d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Validation Accuracy: {:.2f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data.item(),
        accuracy*100))

torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [0/59 (0%)]	Loss: 0.750008
tensor([[0.0010, 0.0240, 0.9510,  ..., 0.5904, 0.6069, 0.6395],
        [0.0010, 0.0020, 1.0000,  ..., 0.5904, 0.6069, 0.6395],
        [0.0000, 0.0000, 1.0000,  ..., 0.5904, 0.6069, 0.6395],
        ...,
        [0.0000, 0.0220, 0.9970,  ..., 0.3796, 0.6338, 0.5088],
        [0.0100, 0.0000, 1.0000,  ..., 0.5904, 0.5965, 0.5304],
        [0.0000, 0.0000, 1.0000,  ..., 0.5904, 0.5965, 0.5304]])
Predictions: tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Actual values: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1.,
        1., 1., 1., 1., 1.])
 Train Epoch: 1/20 

Predictions: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1])
Actual values: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1.,
        1., 1., 1., 1., 1.])
 Train Epoch: 11/20 [59/59 (0%)]	Loss: 0.555599	 Validation Accuracy: 91.53%
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 12/20 [0/59 (0%)]	Loss: 0.542856
tensor([[0.0010, 0.0240, 0.9510,  ..., 0.5904, 0.6069, 0.6395],
        [0.0010, 0.0020, 1.0000,  ..., 0.5904, 0.6069, 0.6395],
        [0.0000, 0.0000, 1.0000,  ..., 0.5904, 0.6069, 0.6395],
        ...,
        [0.0000, 0.0220, 0.9970,  ..., 0.3796, 0.6338, 0.5088],
        [0.0100, 0.0000, 1.0000,  ..., 0.5904, 0.5965, 0.5304],
      

### Testing
* Take testing dataset, run it against current model
* Evaluate testing accuracy

In [6]:
# Follow same steps as when we test with validation set, except with the test set
evaluate_x = Variable(test_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
evaluate_y = Variable(test_loader.dataset.data[:, -1:])

model.eval()
output = model(evaluate_x)
pred = output.data.max(1)[1]

y_labels = torch.flatten(evaluate_y.data)
d = pred.eq(y_labels).cpu()
accuracy = d.sum().item()*1./d.size()[0]  ## Accuracy = (TP+TN)/(TP+TN+FP+FN)

print('\n Testing Accuracy: {:.2f}%'.format(

    accuracy*100))


 Testing Accuracy: 75.00%
